In [9]:
from pathlib import Path
from typing import List

from IPython.display import display
from ipywidgets import widgets

import logging

logger = logging.getLogger()

out = widgets.Output()

from dashboard_funcs import (
    create_start_standalone_command,
    write_shell_file,
    extract_archive,
    s3_download_file,
    s3_list_contents,
    write_fews_desktop_shortcut
)

BUCKET_NAME = "ciroh-rti-hefs-data"
FEWS_BASE_DIR = Path(Path.home(), "fews")
FEWS_INSTALL_DIR = Path("/opt", "fews")

FEWS_BASE_DIR.mkdir(exist_ok=True)

S3_ROOT_DIR = "standalone_configurations"

In [ ]:
# List the unique RFCs in the s3 bucket.
s3_filelist = s3_list_contents(prefix=S3_ROOT_DIR)
unique_rfcs = list(set([Path(filepath).parts[1] for filepath in s3_filelist]))

In [10]:
rfc_dropdown = widgets.Dropdown(
    options=unique_rfcs,
    description='RFC:',
    disabled=False,
    value="ABRFC",
)

In [11]:
def get_rfc_s3_filename(value: str) -> List:
    """Get the list of standalone configuration files for the selected RFC."""
    rfc_s3_filelist = s3_list_contents(prefix=f"{S3_ROOT_DIR}/{rfc_dropdown.value}")
    rfc_s3_filenames = [Path(filepath).name for filepath in rfc_s3_filelist]
    return rfc_s3_filenames


In [12]:
sa_dropdown = widgets.Dropdown(
    options=get_rfc_s3_filename(rfc_dropdown.value),
    description='Standalone:',
    disabled=False,
)

In [13]:
def update_sa_dropdown(*args):
    # Get the SA's for the selected RFC.
    rfc_s3_filelist = s3_list_contents(prefix=f"{S3_ROOT_DIR}/{rfc_dropdown.value}")
    rfc_s3_filenames = [Path(filepath).name for filepath in rfc_s3_filelist]
    sa_dropdown.options = rfc_s3_filenames

In [14]:
rfc_dropdown.observe(update_sa_dropdown, "value")

In [15]:
@out.capture(clear_output=True, wait=True)
def install_fews_standalone(b) -> None:
    """Download standalone configuration from S3 to the working directory."""
    # 1. Download sa from S3
    print("Downloading standalone tar.gz file...")
    selected_sa_s3_filepath = f"{S3_ROOT_DIR}/{rfc_dropdown.value}/{sa_dropdown.value}"
    local_sa_parent_dir = Path(FEWS_BASE_DIR, S3_ROOT_DIR, Path(selected_sa_s3_filepath).stem)
    local_sa_filepath = Path(local_sa_parent_dir, sa_dropdown.value)
    local_sa_parent_dir.mkdir(exist_ok=True, parents=True)
    s3_download_file(
        remote_filepath=selected_sa_s3_filepath,
        local_filepath=local_sa_filepath
    )
    print("Download complete.")
    print("Extracting standalone configuration...")
    # 2. Extract standalone configuration
    extract_archive(local_sa_filepath, local_sa_parent_dir)
    # 3. Create the bash command to run the standalone
    sa_dir_path = list(local_sa_parent_dir.glob("*"))[0]  # TODO: Hacky?
    bash_command_str = create_start_standalone_command(
        FEWS_INSTALL_DIR.as_posix(),
        sa_dir_path
    )
    # 4. Write the command to start FEWS to a shell script
    shell_script_filepath = Path(sa_dir_path, "start_fews_standalone.sh")
    write_shell_file(shell_script_filepath, bash_command_str)
    # 5. Copy in patch file for the downloaded standalone config.
    print("Downloading patch file...")
    s3_download_file(
        "fews-install/fews-NA-202102-125264-patch.jar",
        Path(sa_dir_path, "fews-NA-202102-125264-patch.jar")
    )
    # 6. Create FEWS desktop shortcut that calls the shell script
    desktop_shortcut_filepath = Path(Path.home(), "Desktop", f"{sa_dir_path.name}.desktop")
    print(f"Creating FEWS desktop shortcut...{desktop_shortcut_filepath}")
    write_fews_desktop_shortcut(desktop_shortcut_filepath, shell_script_filepath)
    print("Installation complete.")
    return

In [16]:
config_button = widgets.Button(description='Configure Standalone')
config_button.on_click(install_fews_standalone)

![CIROH Logo](CIROHLogo_200x200.png)

## Welcome to the HEFS-FEWS TEEHR-Hub Dashboard!

This experimental dashboards allows you to download and install pre-existing FEWS stand-alone configurations from the ciroh-rti-hefs-data S3 bucket, and a desktop shortcut to open FEWS.

Configure FEWS for your region of interest:

In [ ]:
display(rfc_dropdown)

In [ ]:
display(sa_dropdown)

In [ ]:
display(config_button)

In [ ]:
display(out)